Load Scanpy packages.

Notes: 
- Scanpy is a scalable toolkit for analyzing single-cell gene expression data built jointly with anndata. It includes preprocessing, visualization, clustering, trajectory inference and differential expression testing. The Python-based implementation efficiently deals with datasets of more than one million cells.
- AnnData is a Python package that defines a data structure designed to efficiently store large data sets like scRNA-seq.

In [26]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 300
sc.logging.print_header()

scanpy==1.10.4 anndata==0.11.3 umap==0.5.7 numpy==1.26.4 scipy==1.13.1 pandas==2.2.2 scikit-learn==1.5.1 statsmodels==0.14.2 pynndescent==0.5.13


Confirm current working directory.

In [29]:
import os
print("Current Working Directory:", os.getcwd())

Current Working Directory: C:\Users\music\OneDrive - Johns Hopkins\Documents\Johns Hopkins- Senior Year\Comp Stem Cell Bio\cscb


Load Sample 1. 

Notes:
- sc.read_10x_h5 function reads the 10x-Genomics-formatted hd5 file and returns an annotated data matrix where cells are named by their barcode and genes by their gene name.

In [34]:
ad10f = sc.read_10x_h5("hw1_data/pbmc_10k_v3_filtered_feature_bc_matrix.h5")
print(ad10f)
type(ad10f)

AnnData object with n_obs × n_vars = 11769 × 33538
    var: 'gene_ids', 'feature_types', 'genome'


anndata._core.anndata.AnnData

In [36]:
ad10f.shape

(11769, 33538)

In [40]:
#confirm .obs is empty and no metadata has been added yet
print(ad10f.obs.columns)

ncells = 3
ngenes = 4
randGenes = np.random.choice(ad10f.var_names, size=ngenes, replace=False)
randCells = np.random.choice(ad10f.obs_names, size=ncells, replace=False)
print(randCells)
print(randGenes)

Index([], dtype='object')
['TACCCACTCCACGTAA-1' 'TCCATCGTCCCGAACG-1' 'GCAGCCACAGACAATA-1']
['GINM1' 'AL109610.1' 'GALT' 'SPOPL']


In [42]:
#Reindexing only valid with uniquely valued Index objects so make unique.

gname_counts = ad10f.var_names.value_counts()

np.any(gname_counts>1)
duplicates = gname_counts[gname_counts > 1]

ad10f.var_names_make_unique()
ad10f[:, randGenes ]
ad10f[randCells, randGenes ]

View of AnnData object with n_obs × n_vars = 3 × 4
    var: 'gene_ids', 'feature_types', 'genome'

In [44]:
# View first few rows of gene metadata
print(ad10f.var.head())

                    gene_ids    feature_types  genome
MIR1302-2HG  ENSG00000243485  Gene Expression  GRCh38
FAM138A      ENSG00000237613  Gene Expression  GRCh38
OR4F5        ENSG00000186092  Gene Expression  GRCh38
AL627309.1   ENSG00000238009  Gene Expression  GRCh38
AL627309.3   ENSG00000239945  Gene Expression  GRCh38


In [46]:
ad10f.obs['sample_name'] = "sample_1"
print(ad10f.obs.columns)
print(ad10f[randCells].obs)

Index(['sample_name'], dtype='object')
                   sample_name
TACCCACTCCACGTAA-1    sample_1
TCCATCGTCCCGAACG-1    sample_1
GCAGCCACAGACAATA-1    sample_1


In [48]:

ad10f[randCells, randGenes ].X

print(ad10f.var.columns)
print(ad10f[:,randGenes].var)

Index(['gene_ids', 'feature_types', 'genome'], dtype='object')
                   gene_ids    feature_types  genome
GINM1       ENSG00000055211  Gene Expression  GRCh38
AL109610.1  ENSG00000225785  Gene Expression  GRCh38
GALT        ENSG00000213930  Gene Expression  GRCh38
SPOPL       ENSG00000144228  Gene Expression  GRCh38


In [50]:
# .obs and .var are Pandas dataframes that can be accessed with .loc and .iloc
print(ad10f.var.loc[randGenes])

                   gene_ids    feature_types  genome
GINM1       ENSG00000055211  Gene Expression  GRCh38
AL109610.1  ENSG00000225785  Gene Expression  GRCh38
GALT        ENSG00000213930  Gene Expression  GRCh38
SPOPL       ENSG00000144228  Gene Expression  GRCh38


1. What is the cell type of composition of PBMCs of Sample 1?

Make a new .obs column indicating cell type.

In [ ]:
#indexes of .obs are cell barcodes
print(ad10f.obs.index[:5]) 

In [54]:
#double checking which marker genes are in the ad10f .var
marker_genes_test = ["CD3D", "CD14", "LYZ", "MS4A1", "GNLY", "NKG7", "KLRC1", "NKG2A"]

# Check which markers are in the dataset
genes_present = [gene for gene in marker_genes_test if gene in ad10f.var_names]
genes_missing = [gene for gene in marker_genes_test if gene not in ad10f.var_names]

print(f"Genes Found: {genes_present}")
print(f"Genes Missing: {genes_missing}")


Genes Found: ['CD3D', 'CD14', 'LYZ', 'MS4A1', 'GNLY', 'NKG7', 'KLRC1']
Genes Missing: ['NKG2A']


In [56]:
#Dictionary of marker genes
marker_genes = {
    "Monocytes": ["LYZ", "CD14", "CD68", "S100A8", "S100A9", "CCR2", "FCGR3A", "CD16", "CD163", "IL1B", "CX3CR1", "CCR5"],
    "NK Cells": ["NCAM1", "KIR2DL1", "KIR2DL4", "KIR3DL1", "KIR3DL2", "KLRC1", "KLRC2", "KLRC3", "KLRK1", "GNLY", "GZMB"],
    "B Cells": ["CD19", "CD79A", "CD79B", "CD20"],
    "T Cells": ["CD3D", "CD3E", "CD3G"],
    "Dendritic Cells": ["FLT3", "CD11C", "CD1C", "CD123", "CLEC4C"]
}
#Initialize a new .obs column for cell type
ad10f.obs['cell_type'] = "Unknown"

cell_scores = {}

#Assign cells based on marker expression
for cell_type, genes in marker_genes.items():
    # Filter for marker genes that exist in the dataset
    present_genes = [gene for gene in genes if gene in ad10f.var_names]
    
    if present_genes:  # If any marker genes exist in the dataset
        # Compute mean expression per cell for these markers
        mean_expression = np.mean(ad10f[:, present_genes].X.toarray(), axis=1)
        
        # Store the mean expression score
        cell_scores[cell_type] = mean_expression

# Convert the cell_scores dictionary into a DataFrame
cell_scores_df = pd.DataFrame(cell_scores, index=ad10f.obs.index)

# Assign each cell to the cell type with the highest expression score
ad10f.obs["cell_type"] = cell_scores_df.idxmax(axis=1)

# Check the number of cells assigned to each type
print(ad10f.obs["cell_type"].value_counts())

cell_type
T Cells            4720
Monocytes          4655
B Cells            1629
NK Cells            747
Dendritic Cells      18
Name: count, dtype: int64
